#### 크롤링(Crawling)
 - 내가 작성한 프로그램에 의해서, 웹 드라이버(브라우저 밴더별로 만든 웹 드라이버(핸들링이 가능한 브라우저)를 엑세스하여 데이터 수집 행위를 하는 것.
 - 재료
    - 웹 드라이버(로컬 PC에 존재)
      - 브라우저를 직접 띠워서 처리하는 방식.
        - 아나콘다에서 진행.
        - Chrome driver : https://chromedriver.chromium.org/downloads => version에 맞는 driver 이용.
        - chrome://version 확인. 
      - 브라우저를 백엔드(일종의 고스트 브라우저)에서 처리하는 방식.(숨김 기능 => 리눅스 환경)
    - 모듈
      - selenium
      - https://selenium-python.readthedocs.io/
      - 크롬 브라우저도 고스트 기능을 사용할수 있어서 리눅스(서버스타일)에서 구동이 가능.
    - 코랩에서는 불가.

- 설치
    - conda install selenium
    - pip install selenium

In [1]:
# 1. 모듈 가져오기.
from selenium import webdriver as wd
import platform, time

In [2]:
# 2. 브라우저 띠우기.
if platform.system() == 'Darwin': # Mac.
    # 맥 PC는 드라이버 있는 위치에서 아래 명령 터미널에서 수행 후 진행.
    # $ xattr -d com.apple.quarantine chromedriver
    driver = wd.Chrome('c:/chromedriver.exe')
    
else: # Window.    
    driver = wd.Chrome('c:/chromedriver.exe')

In [3]:
# 3. 대상 사이트 : 싼 주유소 찾기 오피넷.
target_site = 'http://www.opinet.co.kr/searRgSelect.do'
target_site

'http://www.opinet.co.kr/searRgSelect.do'

In [4]:
# 4. 접속 => get 방식 진행 => 네트워크 속도, 대상 사이트의 응답속도 등을 고려.
# 접속 후에는 충분히 대기. (명시적, 암묵적, time 모듈을 이용하여 그냥 쉬는 방법)
# selenium을 이용한 크롤링 기술은 네트워크 속도가 아주 빠르거나, 일정한 수준을 유지하는 서버에서 진행. 
# 서버 사양도 나름 괜찮아야 함.(단, 고스트를 사용할 때는 관계 없음)
# 1회에 원하는 사이트로 접속하지 않아, 2회 연속 접속을 통해서 원하는 사이트로 강제 진입.
for _ in range(2):
    driver.get( target_site )
    # 5초 대기.
    time.sleep(5) 

In [5]:
# 5. 특정하는 css selector 확인.
# 시도 : #SIDO_NM0 -> select
# 시도의 목록을 획득.
# find_elements_by_css_selector() : 특정 태그 기준으로 특정 태그의 모든 하위 태그 자식들을 찾기.
option_tag = driver.find_elements_by_css_selector('#SIDO_NM0 > option')
len( option_tag )

18

In [6]:
for option in option_tag  :
    # option들 중에서 value가 존재하는 요소만을 가져옴.
    if option.get_attribute('value') :
        
        # 실제 다음 단계에 필요한 정보 value를 추출.
        print(option.get_attribute('value'))

서울특별시
부산광역시
대구광역시
인천광역시
광주광역시
대전광역시
울산광역시
세종특별자치시
경기도
강원도
충청북도
충청남도
전라북도
전라남도
경상북도
경상남도
제주특별자치도


In [7]:
Sidos = [ option.get_attribute('value') for option in driver.find_elements_by_css_selector('#SIDO_NM0 > option')
         if option.get_attribute('value') ]
Sidos

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [8]:
Sido_select = driver.find_element_by_id('SIDO_NM0')

# 서울과 부산만 선택.
for sido in Sidos[:2] :
    print( sido )
    # select 태그의 value값을 지정함으로써, 시도를 선택을 확정하는 행위.
    # 이후 이벤트는 사이트에 미리 설정된 이벤트 처리에 의해 시군구가 반영.
    # 특정 요소에 value를 설정하는 함수.
    # 이벤트 발생 : # 사이트는 내부적으로 ajax를 이용하여 변경된 시도에 일치되는 시군구 정보를 가져옴.
    Sido_select.send_keys( sido ) 
    
    # 5초 대기.
    time.sleep(5)

서울특별시
부산광역시


#### 자동 순환 처리
- 서울
    - 강남구
        3초 ~ 5초
    - 강동구 
    - ...
    - 중량구
- 부산
    - 강서구
    - ...
    - 해운대구

In [9]:
# 시군구 : #SIGUNGU_NM0 -> select

# 우선 3곳만 확인.
for sido in Sidos[:3]  : 
    print('')
    print(f'< {sido} >')
    print('='*15)
    
    # 시도 변경.
    # 시군구를 선택하면 => 화면이 껌벅(form 전송) => DOM tree를 새로 구성.
    # 기존의 객체들은 전부 사라짐 => 매번 구해오기.
    Sido_select = driver.find_element_by_id('SIDO_NM0')
    Sido_select.send_keys( sido )    
    time.sleep(3)
    
    # 시군구 변경.
    Sigungus = [ option.get_attribute('value')
        for option in driver.find_elements_by_css_selector('#SIGUNGU_NM0 > option')
        if option.get_attribute('value') ]
    
    # 시군구를 돌면서 작업 하도록 선택.
    # 뒤에 2곳만 가져옴.
    for sigungu in Sigungus[-3:] :
        print( sigungu )
        # 시군구 select 태그의, 값을 현재 시군구 설정.
        driver.find_element_by_id('SIGUNGU_NM0').send_keys( sigungu )
        
        # 3초 대기.
        time.sleep(3)
        
        # 조회 : #searRgSelect => button <= 클릭 이벤트.
        # # 엑셀 : javascript : fn_excel_download('os_btn') 로 구성.
        # 클릭 이벤트 : #glopopd_excel 
        # 해당 사이트 엑셀 다운로드 버튼 이벤트 클릭.
        driver.find_element_by_id('glopopd_excel').click()


< 서울특별시 >
종로구
중구
중랑구

< 부산광역시 >
영도구
중구
해운대구

< 대구광역시 >
서구
수성구
중구


In [10]:
# 브라우저 닫기. (창닫고, 프로세스 종료.)
# 메모리 방지 차원.
driver.close()
driver.quit()

In [11]:
# Python 프로세스 종료. => Shutdown.
exit()